In [5]:

import io
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.callbacks import CallbackAny2Vec
import os
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams

In [6]:
PATH = "../Models"
DATASETS = "../DataSets"
ARCHIVO = None
MODEL = os.path.join(PATH, "invernadero.bin")

In [7]:
class EpochLogger(CallbackAny2Vec):
    """
    Callback para mostrar el progreso del entrenamiento al final de cada época.
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f"Época {self.epoch + 1}/{model.epochs} finalizada.")
        self.epoch += 1

# 1. Funciones de extracción y preparación de texto
def extraer_texto(archivo, datasets_dir):
    """
    Extrae texto de un archivo específico (.pdf o .txt) o de todos los archivos 
    (.pdf o .txt) en un directorio si 'archivo' es None.
    Devuelve una única cadena con el texto extraído (concatenado si son múltiples archivos).
    """
    
    def _leer_pdf(ruta_completa):
        contenido = ""
        try:
            laparams = LAParams(
                line_margin=0.4, 
                char_margin=3.5, 
                word_margin=0.2, 
                boxes_flow=0.5
            )
            contenido = extract_text(ruta_completa, laparams=laparams)
        except Exception as e:
            print(f"ADVERTENCIA: No se pudo leer el archivo PDF '{ruta_completa}' con pdfminer.six. Causa: {e}")
        return contenido

    def _leer_txt(ruta_completa):
        contenido = ""
        try:
            with open(ruta_completa, "r", encoding="utf-8", errors="ignore") as f:
                contenido = f.read()
        except Exception as e:
            print(f"ADVERTENCIA: No se pudo leer el archivo de texto '{ruta_completa}'. Causa: {e}")
        return contenido

    # Caso 1: Se especifica un archivo específico.
    if archivo:
        ruta_completa = os.path.join(datasets_dir, archivo)
        if os.path.exists(ruta_completa):
            print(f"Leyendo archivo especificado: {ruta_completa}")
            if ruta_completa.lower().endswith('.pdf'):
                return _leer_pdf(ruta_completa)
            elif ruta_completa.lower().endswith('.txt'):
                return _leer_txt(ruta_completa)
            else:
                print(f"ADVERTENCIA: El archivo '{archivo}' no es un PDF ni un TXT. Ignorando.")
        else:
            print(f"ADVERTENCIA: El archivo especificado '{ruta_completa}' no existe.")
        return ""

    # Caso 2: No se especifica un archivo, escanear el directorio completo.
    elif os.path.isdir(datasets_dir):
        print(f"Escaneando y extrayendo texto de todos los archivos .pdf y .txt en: {datasets_dir}")
        textos_extraidos = []
        for nombre_archivo in sorted(os.listdir(datasets_dir)):
            ruta_completa = os.path.join(datasets_dir, nombre_archivo)
            texto_actual = ""
            if nombre_archivo.lower().endswith('.pdf'):
                texto_actual = _leer_pdf(ruta_completa)
            elif nombre_archivo.lower().endswith('.txt'):
                texto_actual = _leer_txt(ruta_completa)
            
            if texto_actual and texto_actual.strip():
                print(f"  Extraído: {nombre_archivo}")
                textos_extraidos.append(texto_actual)
        
        if textos_extraidos:
            print(f"Se extrajo texto de {len(textos_extraidos)} archivo(s) con contenido válido.")
            return "\n".join(textos_extraidos) # Unir todo el texto en una sola cadena.
        else:
            print("ADVERTENCIA: No se encontraron archivos .pdf o .txt con contenido válido en el directorio.")
            return ""
    
    # Caso 3: Ni archivo específico ni directorio válido.
    else:
        print(f"ADVERTENCIA: Ni el archivo '{archivo}' ni el directorio '{datasets_dir}' son válidos.")
        return ""

# 2. Convertir texto en corpus (Optimizado)
def preparar_corpus(texto):
    """
    Prepara un corpus a partir de un bloque de texto usando un generador y StringIO
    para mayor eficiencia de memoria, procesando el texto línea por línea.
    """
    # Usar io.StringIO para tratar la cadena de texto como un archivo en memoria
    text_stream = io.StringIO(texto)
    for linea in text_stream:
        linea_limpia = linea.strip()
        if linea_limpia:
            yield simple_preprocess(linea_limpia)


In [8]:
# --- PROCESO ---
print('Iniciando extracción de texto...')
texto = extraer_texto(ARCHIVO, DATASETS)
with open("inv.txt", "w", encoding="utf-8") as f:
    f.write(texto)

Iniciando extracción de texto...
Escaneando y extrayendo texto de todos los archivos .pdf y .txt en: ../DataSets
  Extraído: 217c0e1e65d907aa506a9515762adff3.pdf
  Extraído: MANUALBASICOPARAELMANEJODEINVERNADEROS.pdf
  Extraído: Manual de Control de Clima.pdf
  Extraído: control_clima_riego_invernadero.txt
  Extraído: guiadeinvernaderos.pdf
  Extraído: inta_-_invernaderos.pdf


Cannot set gray non-stroke color because /'P17223' is an invalid float value
Cannot set gray non-stroke color because /'P17225' is an invalid float value
Cannot set gray non-stroke color because /'P17291' is an invalid float value
Cannot set gray non-stroke color because /'P17293' is an invalid float value
Cannot set gray non-stroke color because /'P17295' is an invalid float value
Cannot set gray non-stroke color because /'P17297' is an invalid float value
Cannot set gray non-stroke color because /'P17299' is an invalid float value


  Extraído: libro-invernaderos-de-mexico.pdf
Se extrajo texto de 7 archivo(s) con contenido válido.


In [9]:
if texto and texto.strip():
    print('Texto extraído. Preparando corpus...')
    corpus = preparar_corpus(texto)
else:
    print('No se pudo extraer texto. El corpus no será generado.')
    corpus = []

Texto extraído. Preparando corpus...


In [10]:
print('Iniciando entrenamiento...')
epoch_logger = EpochLogger()

# Convertir el generador del corpus a una lista para permitir múltiples pasadas
print('Materializando el corpus en memoria para el entrenamiento...')
corpus_list = list(corpus)
print(f'Corpus materializado. {len(corpus_list)} documentos listos para entrenar.')

modelo = Word2Vec(
    sentences=corpus_list, # Usar la lista, no el generador
    sg=1,  # Usar Skip-gram
    vector_size=300,  # Aumentar dimensionalidad
    workers=6,
    window=8,  # Aumentar ventana de contexto
    min_count=1,
    epochs=50,  # Número de épocas más razonable
    negative=10,  # Muestreo negativo
    callbacks=[epoch_logger]
)

Iniciando entrenamiento...
Materializando el corpus en memoria para el entrenamiento...
Corpus materializado. 18403 documentos listos para entrenar.
Época 1/50 finalizada.
Época 2/50 finalizada.
Época 3/50 finalizada.
Época 4/50 finalizada.
Época 5/50 finalizada.
Época 6/50 finalizada.
Época 7/50 finalizada.
Época 8/50 finalizada.
Época 9/50 finalizada.
Época 10/50 finalizada.
Época 11/50 finalizada.


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Época 12/50 finalizada.
Época 13/50 finalizada.
Época 14/50 finalizada.
Época 15/50 finalizada.
Época 16/50 finalizada.
Época 17/50 finalizada.
Época 18/50 finalizada.
Época 19/50 finalizada.
Época 20/50 finalizada.
Época 21/50 finalizada.
Época 22/50 finalizada.
Época 23/50 finalizada.
Época 24/50 finalizada.
Época 25/50 finalizada.
Época 26/50 finalizada.
Época 27/50 finalizada.
Época 28/50 finalizada.
Época 29/50 finalizada.
Época 30/50 finalizada.
Época 31/50 finalizada.


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


Época 32/50 finalizada.
Época 33/50 finalizada.
Época 34/50 finalizada.
Época 35/50 finalizada.
Época 36/50 finalizada.
Época 37/50 finalizada.
Época 38/50 finalizada.
Época 39/50 finalizada.
Época 40/50 finalizada.
Época 41/50 finalizada.
Época 42/50 finalizada.
Época 43/50 finalizada.
Época 44/50 finalizada.
Época 45/50 finalizada.
Época 46/50 finalizada.
Época 47/50 finalizada.
Época 48/50 finalizada.
Época 49/50 finalizada.
Época 50/50 finalizada.


In [11]:
print('\nGuardando modelo...')
modelo.save(os.path.join(PATH, MODEL))
print("Entrenamiento finalizado.")


Guardando modelo...
Entrenamiento finalizado.
